<a href="https://colab.research.google.com/github/MWaser/BurstIQ/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook is for the purposes of
### 1.   Familiarizing the reader with BurstIQ's admin tool cliq2
### 2.   Allowing the developer to create a new interactive API for all BurstIQ operations from the cliq2 source code



## 1. Upgrade Python Version if necessary

The BurstIQ API requires Python 3.13 or greater.  Check the version to see if upgrading is necessary.

In [6]:
!python --version

Python 3.11.11


# If upgrading is necessary, run the next box. <br />
Don't worry about any warnings recommending a virtual machine -- a notebook is already a virtual machine.

In [7]:
!sudo apt-get update -y
!sudo apt-get install python3.13

!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.13 1
print('')
print('')
!python --version
print('')
print('')
!sudo apt-get install python3-pip

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

# 2. Install BurstIQ Command Line (cliq2)

Grab Python Wheel Installation file from Github Repo

In [ ]:
!curl -H 'Accept: application/vnd.github.v3.raw' -O \
  -L https://api.github.com/repos/clinical-squared/BurstIQ/contents/cliq-3.1.0-py3-none-any.whl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   141  100   141    0     0    669      0 --:--:-- --:--:-- --:--:--   671


In [3]:
!curl -O -J https://graph.burstiq.com/dev_docs/pages/cliq/sources/cliq-3.1.0-py3-none-any.whl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36550  100 36550    0     0  76569      0 --:--:-- --:--:-- --:--:-- 76624


Install from Wheel file

In [ ]:
!pip3 --version

pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


In [4]:
!pip3 install --break-system-packages cliq-3.1.0-py3-none-any.whl

Processing ./cliq-3.1.0-py3-none-any.whl
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of cliq to determine which version is compatible with other requirements. This could take a while.
ERROR: Package 'cliq' requires a different Python: 3.11.11 not in '>=3.13.0'


Verify that command line program is installed correctly

In [ ]:
!cliq -h

usage: cliq2 [-h] [-p PROPS_FILE]
             {version,drop_customer,drop_sdz,update_customer,predict,update_dict,import_dbschema,export_dbschema,truncate_chain,load,query,create_smart,execute_smart}
             ...

Command Line IQ (CLIQ) working with LifeGraph (aka GraphChain): Performs many of the admin-centric
commands necessary to support/manage a customer and its SDZs.

positional arguments:
  {version,drop_customer,drop_sdz,update_customer,predict,update_dict,import_dbschema,export_dbschema,truncate_chain,load,query,create_smart,execute_smart}
                        type "<cmd> --help" for help on specific command
    version             version number
    drop_customer       drop a customer by short name, BIQ_ADMIN only
    drop_sdz            drop a customer's sdz by short name, BIQ_ADMIN only
    update_customer     upserts a customer JSON configuration file, BIQ_ADMIN only
    predict             uses a data file to predict the dictionary and mapping
    update_dict      

In [ ]:
!cliq2 update_customer -h

usage: cliq2 update_customer [-h] --file FILE

options:
  -h, --help            show this help message and exit
  --file FILE, -f FILE  the file path for the input


# 3. Build the Config file (cliq2.yml)

Documentation is at https://admin.burstiq.com/docs/pages/metadata_tools/cliq2/cliq2_desc.html#configuration

Example cliq2.yml:
> biq_admin:
>>  username: burstiq.admin  
>>  password: XXXXXXXXXXXXXXXXXX  
>>  server: https://keycloak.app.burstiq.com/auth  
>>  client_id: aaaaaaaaaaaa  
>>  realm: bbbbbbbbbbbb

> sdz_admin:
>>  username: john.wayne  
>>  password: XXXXXXXXXXXXXXXXXX  
>>  server: https://keycloak.app.burstiq.com/auth  
>>  client_id: burst  
>>  realm: { customer_shortname }  

> graphchain:
>>  server: https://app.burstiq.com  
>>  customer: { customer_shortname }  
>>  sdz: { customer_sdz_name }  

Use secrets for username, password, customer_shortname and customer_sdz name (the key on the left menu) to generate cliq.yml <br />
<b>IMPORTANT!!! </b> If you uncomment the #cat, be sure to delete the results before saving or all your secret information will be in the save file!

In [ ]:
from google.colab import userdata
yml = open(r"cliq2.yml", "w")
yml.write("sdz_admin:\n");
yml.writelines("   username: " + userdata.get('username') + "\n");
yml.writelines("   password: " + userdata.get('password') + "\n");
yml.writelines("   server: https://keycloak.app.burstiq.com/auth\n");
yml.writelines("   client_id: burst\n");
yml.writelines("   realm: " + userdata.get('customer_shortname') + "\n");
yml.writelines("graphchain:\n");
yml.writelines("   server: https://app.burstiq.com\n");
yml.writelines("   customer: " + userdata.get('customer_shortname') + "\n");
yml.writelines("   sdz: " + userdata.get('customer_sdz_name') + "\n");
yml.close();

!cat cliq2.yml


# 4. Initialize the new API

Copy Cliq2 Wheel file and unzip original BurstIQ source files from it

In [ ]:
!curl -H 'Accept: application/vnd.github.v3.raw' -O \
  -L https://api.github.com/repos/clinical-squared/BurstIQ/contents/cliq2-1.0.0-py3-none-any.whl

! cp cliq2-1.0.0-py3-none-any.whl API.zip
! unzip -o API.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21896  100 21896    0     0   114k      0 --:--:-- --:--:-- --:--:--  114k
Archive:  API.zip
  inflating: __init__.py             
  inflating: cliq2.py                
  inflating: cliq_properties.py      
  inflating: consts.py               
  inflating: util.py                 
  inflating: actions/__init__.py     
  inflating: actions/cliq_action.py  
  inflating: actions/create_smart_contract.py  
  inflating: actions/drop_customer.py  
  inflating: actions/drop_sdz.py     
  inflating: actions/execute_smart_contract.py  
  inflating: actions/export_dbschema.py  
  inflating: actions/import_dbschema.py  
  inflating: actions/load_data.py    
  inflating: actions/predict.py      
  inflating: actions/query.py        
  inflating: actions/truncate_chain.py  
  inflating: actions/update_customer.py  
  inflating: actions/up

Import and validate CliqProperties

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.11/dist-packages/')
print("IGNORE clint SYNTAX warning (if clint has not been imported before)")
import clint
from cliq_properties import CliqProperties
from util import Util
props = CliqProperties(Util.standardize_file("cliq2.yml"))
print(props._graphchain)

IGNORE clint SYNTAX warning (if clint has not been imported before)
{'server': 'https://app.burstiq.com', 'customer': 'clinical_squared', 'sdz': 'dev1'}


Install and validate upgraded /graphchain_client/graphchain_client.py

In [ ]:
!curl -H 'Accept: application/vnd.github.v3.raw' -O \
  -L https://api.github.com/repos/clinical-squared/BurstIQ/contents/graphchain_client/graphchain_client.py
!mv graphchain_client.py graphchain_client
from actions.cliq_action import CliqAction
print ("CliqAction imported!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7915  100  7915    0     0  37093      0 --:--:-- --:--:-- --:--:-- 37159
CliqAction imported!


# 5. Start Testing/Using the API

Grab an access token for future API calls

In [ ]:
realm = userdata.get('customer_shortname')
un = userdata.get('username')
pw = userdata.get('password')
print(un)
print(pw)
print(realm)
adminToken = CliqProperties._get_token("https://keycloak.app.burstiq.com/auth", "burst", realm, un, pw)
print(adminToken)

mark.waser@clinicalsquared.com
lifeH34d!
clinical_squared


RuntimeError: error 401: resp: {"error":"invalid_grant","error_description":"Invalid user credentials"}

Since we need to build out our NPPES Data Dictionary, let's try the Predict function.

In [ ]:
!curl -H 'Accept: application/vnd.github.v3.raw' -O \
  -L https://api.github.com/repos/clinical-squared/BurstIQ/contents/NP_Providers_100.csv
from graphchain_client.graphchain_client import GraphChainClient
cust = userdata.get('customer_shortname')
sdz = userdata.get('customer_sdz_name')
gcc = GraphChainClient('https://app.burstiq.com', adminToken, cust, sdz)
dictionary, mapping = gcc.predict("NP_Providers_100.csv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 69629  100 69629    0     0   279k      0 --:--:-- --:--:-- --:--:--  280k


RuntimeError: error 500: resp: {"id":"ae244b37-88fc-417c-a2a6-ca960c2e2f05","timestamp":"2024-01-17T13:05:49.763Z","status":500,"error":"INTERNAL_SERVER_ERROR"}

Yeah, that's unfortunate.  <br />Let's verify that Cliq2 can't handle the file either rather than it being an API problem.

In [ ]:
!cliq2 predict -f "NP_Providers_100.csv"

Traceback (most recent call last):
  File "/usr/local/bin/cliq2", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/cliq2.py", line 90, in main
    a.run(opts, props)
  File "/usr/local/lib/python3.11/dist-packages/actions/predict.py", line 21, in run
    dictionary, mapping = gc.predict(opts.file)
                          ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/graphchain_client/graphchain_client.py", line 95, in predict
    code, resp = self._hc.post_multipart(
                 ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/graphchain_client/http_client.py", line 193, in post_multipart
    return self._process_response(resp)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/graphchain_client/http_client.py", line 233, in _process_response
    raise RuntimeError(
RuntimeError: error 500: resp: {"id":"dba64f2c-0ebb-4738-8615-d67ac9033

Yep.  It's BurstIQ's problem.  Reported to Tyson.<br />
Tyson acknowledged the problem and fixed it but it hasn't been rolled to production.<br />We'll try again later.